# Excel to TSV Converter
Upload an `.xlsx` file, and this notebook will extract a specific sheet and convert it to a **tab-separated** format (TSV).

In [55]:
import pandas as pd
from io import StringIO
from IPython.display import display, HTML
import ipywidgets as widgets

In [56]:
# Upload widget
print("BOB")
upload = widgets.FileUpload(accept='.xlsx', multiple=False)
display(upload)

BOB


FileUpload(value=(), accept='.xlsx', description='Upload')

In [ ]:
def handle_upload(change):
    if upload.value:
        uploaded_file = upload.value[0]
        content = uploaded_file['content']
        filename = uploaded_file.get('name', 'uploaded.xlsx')  # fallback filename

        # Load the Excel file
        import io
        xls = pd.ExcelFile(io.BytesIO(content))

        print(f"Uploaded: {filename}")
        print("Sheets found:", xls.sheet_names)

        # Choose the first sheet
        df = xls.parse(xls.sheet_names[0],dtype={'pdga_number': str})
        # Keep only relevant columns
        df = df[['division', 'name', 'pdga_number']].copy()

        # Remove emoji/symbols like ✅ and strip extra whitespace
        df['name'] = df['name'].str.replace('✅', '', regex=False).str.strip()

        # Split so that the last word is the last name, the rest is first name
        df[['firstname', 'lastname']] = df['name'].str.extract(r'^(.*?)\s+(\S+)$')

        # Drop original name column
        df = df.drop(columns=['name'])

        # Reorder columns if needed
        df = df[['firstname', 'lastname', 'pdga_number', 'division']]

        # Convert to TSV
        tsv = df.to_csv(sep='\t', index=False)
        print("\nTSV Output (first 500 characters):\n")
        display(HTML(f"<pre>{tsv}</pre>"))



In [58]:
upload.observe(handle_upload, names='value')